# Backtesting with backtrader library
Notes:  
Timestamp must be in the exact format specified for datafeed   **Timestamp needs to be first column index**  
Params for datafeed need to be specified as tuple of tuples  
Within strategy data is accessed via self.datas[0] for ex: self.datas[0].high_delta[0] would get you the first timeperiod's high_delta  
The current data has already happened and cannot be used to execute an order (ex. you cannot look at the current close price and simultaneously buy the current close, the order will be executed at open instead), orders will be executed on the following day.

In [1]:
import backtrader as bt
import datetime
import pandas as pd

## Define Strategies and Data utils

In [2]:
class BasicStrategy(bt.Strategy):
    def __init__(self):
        # To keep track of pending orders
        self.order = None
        print('***The limit sells for this strategy are valid for 1 day***')
        
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        elif order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
            
            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        # set no pending order
        self.order = None


    def next(self):
        # STRATEGY 
        data = self.datas[0]
        self.log(f'Current Portfolio Value : {self.broker.get_value()}')
        
        # cancel if there is an order pending, this strategy should have 1 working order per day
        if self.order:
            self.log('ORDER CANCELLED')
            self.cancel(self.order)
        
        # Check if we are in the market
        if not self.position:
            # BUY
            try:
                self.size = int(self.broker.get_cash() / self.datas[0].open[1])
            except:
                print('Size Exception. If at the end of data, ignore.')
            # invest if prediction looks good
            self.log(f'MARKET BUY CREATE {self.size} shares at next open, current close price: {data.close[0]}')
            self.buy(size=self.size) # market order buys at next open                      
                
        else:
            # place sell order at predicted high if predicted high is greater than current close price
            # TODO: Make prediction and close a filter in the class constructor (more optimal)
            if data.prediction[0] >= data.close[0]:
                self.log(f'LIMIT SELL CREATE {self.size} shares at {data.prediction[0]}')  
                self.sell(exectype=bt.Order.Limit,
                             price=data.prediction[0],
                             valid=data.datetime.date(0) + datetime.timedelta(days=2),
                             size=self.size)
        # if prediction is less than current value sell at open (ASAP)
            else:
                self.log('MARKET SELL CREATE. PREDICTION < CURRENT CLOSE')
                self.sell(size=self.size)
        

In [3]:
class BuyAndHold(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

      
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.datas[0].open[1])
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print(f'Stop price: {self.datas[0].close[0]}')
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [4]:
def prepare_data(data, fromdate, todate, filepath):
    """Prepare data for backtrader datafeed object
        Returns prepared data filepath and params for the GenericCSVData class, also returns columns used"""
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    # start param setup for backtrader
    start = data['timestamp'].iloc[0]
    end = data['timestamp'].iloc[-1]
    from_to = [(start.year, start.month, start.day), (end.year, end.month, end.day)]
    # Backtrader string format
    data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # backtrader data feed class needs a file path, this could be a temp file that's constantly overwritten
    data.to_csv(filepath)
    
    starting_params = [
    ('fromdate', datetime.datetime(*from_to[0])),
    ('todate', datetime.datetime(*from_to[1])),
    ('nullvalue', 0.0),
    ('dtformat', ('%Y-%m-%d %H:%M:%S')),
    ('tmformat', ('%H:%M:%S')),
    ('datetime', 1)]
    # skip nonfeatures (timestamp)
    cols = data.columns[1:]
    # get column position for each indicator and add to starting params list
    i = 2 # starting index since others are reserved
    for indicator in cols:
        starting_params.append((indicator, i))
        i+=1
    final_params = tuple(starting_params)

    return filepath, final_params, cols 




## Import Data and Run Backtest

In [5]:
import os

In [6]:
def capstone_data_prep(pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000):
    """Main loop, preps data and executes backtrader on all stocks in the prediction folder"""
    ticker_backtesting_dict = {}
    for pred_file in os.listdir(f'../../data/ticker_predictions/{pred_folder_name}'):
        # collect data
        ticker_name = pred_file.split('_')[0]
        pred_df = pd.read_csv(f'../../data/ticker_predictions/{pred_folder_name}/{pred_file}')
        ticker_df = pd.read_csv(ticker_df_filepath + f'/{ticker_name}_full_data.csv')
        
        # make timestamp in column position 0, SPECIFIC TO FULL DATA IN FILE 
        ticker_df = ticker_df.rename({'reportperiod':'timestamp'}, axis=1)
        ticker_cols = list(ticker_df.columns)
        ticker_cols[5] = 'ts_cpy'
        ticker_cols[0] = 'timestamp'
        ticker_df.columns = ticker_cols
        ticker_df['timestamp'] = ticker_df['ts_cpy']
        # merge prediction data with full data, on key = timestamp
        ticker_df = ticker_df.merge(pred_df.loc[:, ['timestamp', pred_col_name]], on='timestamp')
        # new prediction column name = "prediction", important because backtester strategy looks for this column name
        ticker_df = ticker_df.rename({pred_col_name:'prediction'}, axis=1)
        # remove all non-prediction rows in prediction field
        ticker_df = ticker_df[ticker_df.prediction > 0]
        # skip nonfeatures, keep timestamp, SPECIFIC TO FULL DATA IN FILE 
        nonfeatures = ticker_df.columns[1:7]
        features = [col for col in ticker_df.columns if col not in nonfeatures]
        # get index of start and end dates for trading
        ticker_df = ticker_df[features]
        ticker_df.columns = [c.lower() for c in ticker_df.columns]
        idx1 = ticker_df.prediction[ticker_df.prediction > 0].index[0]
        idx2 = ticker_df.prediction[ticker_df.prediction > 0].index[-1]
        start_date = pred_df.timestamp[idx1]
        end_date  = pred_df.timestamp[idx2]
        
        # Prepare data for backtrader
        prep_data = prepare_data(ticker_df.copy(), start_date, end_date, 'temp/prep_data.csv')
        class DataFeed(bt.feeds.GenericCSVData):
            lines = tuple(prep_data[2])
            params = prep_data[1]
        
        print(f"--------Ticker Name-------: {ticker_name}")
        ##### BUY AND HOLD #######
        print('BUY AND HOLD')
        # initialize everything and run strategy
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        #cerebro.addsizer(bt.sizers.FixedSize, stake=10)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BuyAndHold)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        buy_hold = cerebro.run()
        buy_hold_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(buy_hold[0].analyzers.sharpe.get_analysis())
        print(buy_hold[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(buy_hold[0].analyzers.returns.get_analysis())

        #### MAIN STRATEGY #####
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BasicStrategy)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        back = cerebro.run()
        strat_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(back[0].analyzers.sharpe.get_analysis())
        print(back[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(back[0].analyzers.returns.get_analysis())
        print(f"For {ticker_name}, return was {buy_hold_ret*100}% for buy and hold vs {strat_ret*100}%  for strategy")
        print('\n'*5)

        ticker_backtesting_dict[ticker_name] = cerebro
        
    return ticker_backtesting_dict

In [7]:
# pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000

cerebro_dict = capstone_data_prep('ar_preds', 'prediction', '../../data/ticker_data')
cerebro_dict

--------Ticker Name-------: MSFT
BUY AND HOLD
Starting Portfolio Value: 100000
2019-09-18, BUY EXECUTED, 137.36
Stop price: 336.989990234375
ROI:        145.33%
Final Portfolio Value: 245330.63244628906

STRATEGY INFO:
OrderedDict([('sharperatio', 0.5993527394373089)])
AutoOrderedDict([('len', 0), ('drawdown', 0.0), ('moneydown', 0.0), ('max', AutoOrderedDict([('len', 98), ('drawdown', 28.234899394065955), ('moneydown', 38787.839111328125)]))])
OrderedDict([('rtot', 0.897436634961623), ('ravg', 0.0016557871493756882), ('rnorm', 0.5177946021891543), ('rnorm100', 51.77946021891543)])
Starting Portfolio Value: 100000
***The limit sells for this strategy are valid for 1 day***
2019-09-17, Current Portfolio Value : 100000.0
2019-09-17, MARKET BUY CREATE 728 shares at next open, current close price: 137.38999938964844
2019-09-18, BUY EXECUTED, 137.36
2019-09-18, Current Portfolio Value : 100844.48266601562
2019-09-18, LIMIT SELL CREATE 728 shares at 139.52539573803514
2019-09-19, SELL EXECUT

2020-03-13, SELL EXECUTED, 147.50
2020-03-13, Current Portfolio Value : 108688.22891870543
2020-03-13, MARKET BUY CREATE 776 shares at next open, current close price: 158.8300018310547
2020-03-16, BUY EXECUTED, 140.00
2020-03-16, Current Portfolio Value : 105134.14749780699
2020-03-16, LIMIT SELL CREATE 776 shares at 136.4024438319339
2020-03-17, SELL EXECUTED, 140.00
2020-03-17, Current Portfolio Value : 108688.22891870543
2020-03-17, MARKET BUY CREATE 787 shares at next open, current close price: 146.57000732421875
2020-03-18, BUY EXECUTED, 138.00
2020-03-18, Current Portfolio Value : 110577.02411523863
2020-03-18, LIMIT SELL CREATE 787 shares at 141.4185684567583
2020-03-19, SELL EXECUTED, 142.77
2020-03-19, Current Portfolio Value : 112442.22228113218
2020-03-19, MARKET BUY CREATE 770 shares at next open, current close price: 142.7100067138672
2020-03-20, BUY EXECUTED, 146.00
2020-03-20, Current Portfolio Value : 105781.72698083922
2020-03-20, LIMIT SELL CREATE 770 shares at 138.34

2020-09-02, BUY EXECUTED, 227.97
2020-09-02, Current Portfolio Value : 167571.42916045265
2020-09-02, LIMIT SELL CREATE 723 shares at 233.3305694016695
2020-09-03, Current Portfolio Value : 157196.38577971532
2020-09-03, LIMIT SELL CREATE 723 shares at 218.87647217339577
2020-09-04, Current Portfolio Value : 154991.23357329442
2020-09-04, LIMIT SELL CREATE 723 shares at 215.80434194462703
2020-09-08, Current Portfolio Value : 146611.66622099947
2020-09-08, LIMIT SELL CREATE 723 shares at 204.13026244479445
2020-09-09, SELL EXECUTED, 207.60
2020-09-09, Current Portfolio Value : 150183.2879861362
2020-09-09, MARKET BUY CREATE 703 shares at next open, current close price: 211.2899932861328
2020-09-10, BUY EXECUTED, 213.40
2020-09-10, Current Portfolio Value : 144538.19884429045
2020-09-10, LIMIT SELL CREATE 703 shares at 206.8599143590941
2020-09-11, SELL EXECUTED, 207.20
2020-09-11, Current Portfolio Value : 145824.6901315219
2020-09-11, MARKET BUY CREATE 713 shares at next open, current

2021-01-25, BUY EXECUTED, 229.12
2021-01-25, Current Portfolio Value : 132832.61725513532
2021-01-25, LIMIT SELL CREATE 578 shares at 231.19519413356923
2021-01-26, SELL EXECUTED, 231.86
2021-01-26, Current Portfolio Value : 134179.35831348496
2021-01-26, MARKET BUY CREATE 563 shares at next open, current close price: 232.3300018310547
2021-01-27, BUY EXECUTED, 238.00
2021-01-27, Current Portfolio Value : 131308.0548772057
2021-01-27, LIMIT SELL CREATE 563 shares at 234.58963790776417
2021-01-28, SELL EXECUTED, 235.61
2021-01-28, Current Portfolio Value : 132833.78865711292
2021-01-28, MARKET BUY CREATE 562 shares at next open, current close price: 238.92999267578125
2021-01-29, BUY EXECUTED, 235.99
2021-01-29, Current Portfolio Value : 130568.92934314806
2021-01-29, LIMIT SELL CREATE 562 shares at 233.64283130155133
2021-02-01, Current Portfolio Value : 134890.70213977894
2021-02-01, LIMIT SELL CREATE 562 shares at 241.3886078406754
2021-02-02, SELL EXECUTED, 241.39
2021-02-02, Curren

2021-06-25, Current Portfolio Value : 137662.31500991646
2021-06-25, LIMIT SELL CREATE 519 shares at 266.9426573221365
2021-06-28, Current Portfolio Value : 139582.62134536568
2021-06-28, LIMIT SELL CREATE 519 shares at 270.66951239576764
2021-06-29, SELL EXECUTED, 270.67
2021-06-29, Current Portfolio Value : 140594.41764522417
2021-06-29, MARKET BUY CREATE 519 shares at next open, current close price: 271.3999938964844
2021-06-30, BUY EXECUTED, 270.69
2021-06-30, Current Portfolio Value : 140703.4032104097
2021-06-30, LIMIT SELL CREATE 519 shares at 272.8653204930422
2021-07-01, Current Portfolio Value : 141066.70954585893
2021-07-01, LIMIT SELL CREATE 519 shares at 273.5704111520461
2021-07-02, SELL EXECUTED, 273.57
2021-07-02, Current Portfolio Value : 142089.34976604625
2021-07-02, MARKET BUY CREATE 511 shares at next open, current close price: 277.6499938964844
2021-07-06, BUY EXECUTED, 278.03
2021-07-06, Current Portfolio Value : 141900.28226116343
2021-07-06, LIMIT SELL CREATE 5

...

2020-03-06, SELL EXECUTED, 284.93
2020-03-06, Current Portfolio Value : 118915.34751462442
2020-03-06, MARKET BUY CREATE 449 shares at next open, current close price: 283.8699951171875
2020-03-09, BUY EXECUTED, 264.46
2020-03-09, Current Portfolio Value : 122947.37244748574
2020-03-09, LIMIT SELL CREATE 449 shares at 275.3427827891656
2020-03-10, SELL EXECUTED, 279.72
2020-03-10, Current Portfolio Value : 125767.09189939004
2020-03-10, MARKET BUY CREATE 457 shares at next open, current close price: 280.0
2020-03-11, BUY EXECUTED, 274.76
2020-03-11, Current Portfolio Value : 127147.22687863809
2020-03-11, LIMIT SELL CREATE 457 shares at 279.7129797548625
2020-03-12, Current Portfolio Value : 114639.13911008339
2020-03-12, LIMIT SELL CREATE 457 shares at 252.15252571300317
2020-03-13, SELL EXECUTED, 260.99
2020-03-13, Current Portfolio Value : 119474.19297360879
2020-03-13, MARKET BUY CREATE 489 shares at next open, current close price: 272.0400085449219
2020-03-16, BUY EXECUTED, 243.92


2020-09-01, Current Portfolio Value : 148105.67285155755
2020-09-01, LIMIT SELL CREATE 473 shares at 315.00686905224245
2020-09-02, SELL EXECUTED, 315.01
2020-09-02, Current Portfolio Value : 149135.33826458658
2020-09-02, MARKET BUY CREATE 463 shares at next open, current close price: 320.239990234375
2020-09-03, BUY EXECUTED, 322.00
2020-09-03, Current Portfolio Value : 146463.8333512565
2020-09-03, LIMIT SELL CREATE 463 shares at 318.43055313416346
2020-09-04, Current Portfolio Value : 144505.33826458658
2020-09-04, LIMIT SELL CREATE 463 shares at 314.1711068724413
2020-09-08, Current Portfolio Value : 142306.08826458658
2020-09-08, LIMIT SELL CREATE 463 shares at 309.3880531620436
2020-09-09, SELL EXECUTED, 309.68
2020-09-09, Current Portfolio Value : 143431.1748734733
2020-09-09, MARKET BUY CREATE 458 shares at next open, current close price: 312.0199890136719
2020-09-10, BUY EXECUTED, 312.73
2020-09-10, Current Portfolio Value : 139707.63263714517
2020-09-10, LIMIT SELL CREATE 45

2021-02-26, SELL EXECUTED, 331.16
2021-02-26, Current Portfolio Value : 151066.95209789948
2021-02-26, MARKET BUY CREATE 451 shares at next open, current close price: 332.2200012207031
2021-03-01, BUY EXECUTED, 334.36
2021-03-01, Current Portfolio Value : 151197.74595165925
2021-03-01, LIMIT SELL CREATE 451 shares at 336.9787147349815
2021-03-02, SELL EXECUTED, 336.98
2021-03-02, Current Portfolio Value : 152247.99904982146
2021-03-02, MARKET BUY CREATE 456 shares at next open, current close price: 334.6400146484375
2021-03-03, BUY EXECUTED, 333.32
2021-03-03, Current Portfolio Value : 152042.7934834152
2021-03-03, LIMIT SELL CREATE 456 shares at 335.18632952112483
2021-03-04, SELL EXECUTED, 335.19
2021-03-04, Current Portfolio Value : 153099.04197161063
2021-03-04, MARKET BUY CREATE 456 shares at next open, current close price: 333.8099975585937
2021-03-05, BUY EXECUTED, 335.03
2021-03-05, Current Portfolio Value : 158602.96531145438
2021-03-05, LIMIT SELL CREATE 456 shares at 349.515

2021-07-22, Current Portfolio Value : 178505.15246737545
2021-07-22, LIMIT SELL CREATE 429 shares at 418.66319760502586
2021-07-23, SELL EXECUTED, 418.66
2021-07-23, Current Portfolio Value : 179746.3389530663
2021-07-23, MARKET BUY CREATE 431 shares at next open, current close price: 417.7000122070313
2021-07-26, BUY EXECUTED, 416.83
2021-07-26, Current Portfolio Value : 178328.35842328114
2021-07-26, LIMIT SELL CREATE 431 shares at 416.4176994249903
2021-07-27, Current Portfolio Value : 179000.71737103505
2021-07-27, LIMIT SELL CREATE 431 shares at 417.98855250095727
2021-07-28, Current Portfolio Value : 176444.89052777333
2021-07-28, LIMIT SELL CREATE 431 shares at 412.0172949282147
2021-07-29, SELL EXECUTED, 412.02
2021-07-29, Current Portfolio Value : 177672.06885448034
2021-07-29, MARKET BUY CREATE 432 shares at next open, current close price: 412.0799865722656
2021-07-30, BUY EXECUTED, 410.76
2021-07-30, Current Portfolio Value : 178302.7851630741
2021-07-30, LIMIT SELL CREATE 4

{'MSFT': <backtrader.cerebro.Cerebro at 0x7fada0481e50>,
 'ADSK': <backtrader.cerebro.Cerebro at 0x7fad8139f8d0>,
 'XOM': <backtrader.cerebro.Cerebro at 0x7fad70e4bc90>,
 'WAT': <backtrader.cerebro.Cerebro at 0x7fad71a28650>,
 'HD': <backtrader.cerebro.Cerebro at 0x7fad9087a3d0>,
 'UNH': <backtrader.cerebro.Cerebro at 0x7fad98dff710>}

### Library additionally has plotting
Not sure how to reformat this, it's pretty ugly

In [8]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams.update({'font.size': 12}) 
cerebro_dict['UNH'].plot()

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/opt/anaconda3/lib/python3.7/site-packages/matplotlib/dates.py)

### Ways to Improve:

Increase strategy trading volume. There are certain periods where sell orders are not going through for multiple days in a row, meaning that the price is lower than anticipated by model for that period. For example, if the last sell order didn't go through, we can find a way to use that information to inform our next sell order and make the prediction lower or try a market sell. Maybe train a "bear" model, one that only knows pain and suffering of bear markets, and only produces pessimistic high price predictions.   

Smarter entry points. Currently, buy orders are being executed at market price at the open. Returns may increase if entry constraints are increased.  

Resizing orders. Orders for this strategy baseline are currently 100% in or 100% out of position. This has beneifits if you want less exposure to the long term movement of the stocks. Depending on the situation and strategy, we might want to reserve a cash position to expose ourselves less to short term movements.  